In [1]:
import glob, json, os
import numpy as np

In [2]:
def readJson(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

def invTransform(extrinsics):
    assert extrinsics.shape == ((4,4))
    
    rotation = extrinsics[:3,:3]
    translation = extrinsics[:3,3].T
    
    invExtrinsics = np.identity(4)
    invExtrinsics[:3,:3] = rotation.T
    invExtrinsics[:3,3] = - rotation.T @ translation
    
    assert np.isclose(np.linalg.det(invExtrinsics[:3,:3]), 1.0)
    return invExtrinsics

In [3]:
# IRON wants normalized version
kai_cameras = '/home/leyinghu/Documents/data/Real_data07182022/downscaled_diffuse/colmap_nerfplusplus/posed_images/kai_cameras_normalized.json'
cam_dict = os.path.join(os.path.dirname(kai_cameras), 'cam_dict_norm.json')

In [4]:
data = readJson(kai_cameras)
for frame in data.keys():
    W2C = np.array(data[frame]['W2C']).reshape((4,4))
    invExtrinsics = invTransform(W2C)
    data[frame]['W2C'] = list(invExtrinsics.flatten())

In [5]:
with open(cam_dict, 'w') as f:
    json.dump(data, f, indent=2, sort_keys=True)